# CST 8502 FINAL PROJECT

By:  
Charles-Antoine Campeau  
Joshua Ayyasamy  
Mubarak husain Shaikh  
Curtis Sloan 

Submitted to Dr. Anu Thomas in partial fulfillment of the requirements of CST 8502 

Algonquin College Artificial Intelligence Software Development

2023-11-26

In [ ]:
import pandas as pd
import numpy as np
import math
import re
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import seaborn as sns


## DATA PREPARATION

In [ ]:
# Import the CSV
bicycleTheftData = pd.read_csv("bicycle-thefts - 4326.csv")

In [ ]:
# Remove the unnecessary attributes
bicycleTheftData = bicycleTheftData.drop(columns=["_id","EVENT_UNIQUE_ID", "OCC_DATE", "OCC_YEAR", "OCC_DAY",
                                                   "OCC_DOY", "REPORT_DATE", "REPORT_YEAR", "REPORT_MONTH", "REPORT_DOW", "REPORT_DAY", 
                                                     "REPORT_DOY", "REPORT_HOUR", "LOCATION_TYPE", "BIKE_MODEL", "STATUS", "geometry"])

## DATA CLEANING & CONSTRUCTION
The data cleaning and construction was separated between all members.  Everyone's contribution is indicated

### Charles

In [ ]:
# Import the CSV with the BIKE_MAKE changes
makesData = pd.read_csv("MAKESREPLACEMENT.CSV", delimiter=";")
# Set the index as the good values and convert the strings to list
makesData.set_index("NEW", inplace=True)
makesData["OLD"] = makesData["OLD"].str.split(",")

# Convert the list to a series
makesSeries = makesData["OLD"].explode()

In [ ]:
def GetMake(make):
    """Correct wrongfully inputted bicycle makes \n
    Parameter:
    ------------
    make: The instance bicycle make

    Return:
    -----------
    The properly written make
    """
    
    try:
        index = makesSeries.index[makesSeries.str.contains(re.escape(make))][0]
        return index
    except: 
        return make

In [ ]:
# Fix the errors in the BIKE_MAKE attribute
bicycleTheftData["BIKE_MAKE"] = bicycleTheftData["BIKE_MAKE"].apply(GetMake)

In [ ]:
# Create the dataframe for the clustering and add the BIKE_MAKE and DIVISION - Charles
kMeanClusteringData = pd.DataFrame()
kMeanClusteringData["BIKE_MAKE"] = bicycleTheftData["BIKE_MAKE"]
kMeanClusteringData["DIVISION"] = bicycleTheftData["DIVISION"]

### Joshua

In [ ]:
bicycleTheftData['BIKE_SPEED'] = bicycleTheftData['BIKE_SPEED'].fillna(bicycleTheftData['BIKE_SPEED'].mean())

In [ ]:
low = bicycleTheftData['BIKE_COST'].quantile(.25)
average = bicycleTheftData['BIKE_COST'].quantile(.5)
high = bicycleTheftData['BIKE_COST'].quantile(.75)
bicycleTheftData['BIKE_COST_CATEGORY'] = np.select(
    [
        bicycleTheftData['BIKE_COST'].isna(),
        bicycleTheftData['BIKE_COST'] <= low,
        (bicycleTheftData['BIKE_COST'] > low) & (bicycleTheftData['BIKE_COST'] <= average),
        (bicycleTheftData['BIKE_COST'] > average) & (bicycleTheftData['BIKE_COST'] <= high),
        bicycleTheftData['BIKE_COST'] > high
    ],
    [
        'NK',
        'Low',
        'Average',
        'High',
        'Luxury'
    ],
    default='Unknown'
)

In [ ]:
bicycleTheftData = bicycleTheftData.drop(['BIKE_COST'], axis =1)

In [ ]:
#Binning of BIKE_SPEED
low = bicycleTheftData['BIKE_SPEED'].quantile(.25)
average = bicycleTheftData['BIKE_SPEED'].quantile(.5)
high = bicycleTheftData['BIKE_SPEED'].quantile(.75)
bicycleTheftData['BIKE_SPEED_CATEGORY'] = np.select(
    [
        bicycleTheftData['BIKE_SPEED'].isna(),
        bicycleTheftData['BIKE_SPEED'] <= low,
        (bicycleTheftData['BIKE_SPEED'] > low) & (bicycleTheftData['BIKE_SPEED'] <= average),
        (bicycleTheftData['BIKE_SPEED'] > average) & (bicycleTheftData['BIKE_SPEED'] <= high),
        bicycleTheftData['BIKE_SPEED'] > high
    ],
    [
        'NK',
        'Slow-Speeding',
        'Average-Speeding',
        'Fast-Speeding',
        'Racing'
    ],
    default='Unknown'
)
bicycleTheftData = bicycleTheftData.drop(['BIKE_SPEED'], axis =1)


In [ ]:
# Add the BIKE_COST_CATEGORY and the BIKE_SPEED_CATEGORY to the clustering dataframe - Charles
kMeanClusteringData["BIKE_COST_CATEGORY"] = bicycleTheftData["BIKE_COST_CATEGORY"]
kMeanClusteringData["BIKE_SPEED_CATEGORY"] = bicycleTheftData["BIKE_SPEED_CATEGORY"]

### Mubarak

In [ ]:
# Replace missing values in PRIMARY_OFFENCE with 'Unknown'
bicycleTheftData['PRIMARY_OFFENCE'].fillna('Unknown', inplace=True)

# Convert entries to lowercase for uniformity
bicycleTheftData['PRIMARY_OFFENCE'] = bicycleTheftData['PRIMARY_OFFENCE'].str.lower()

In [ ]:
#there is one instance colored as '18' so to handle such anomalies we will replace it with unknown
bicycleTheftData['BIKE_COLOUR'] = bicycleTheftData['BIKE_COLOUR'].replace('18', 'Unknown')

bicycleTheftData['BIKE_COLOUR'].fillna('Unknown', inplace=True)

In [ ]:
# Function to categorize colors
def categorize_color(color):
    color = color.lower()  # Convert to lowercase for uniformity

    color_categories = {
        'black': ['black', 'blk', 'blac'],
        'blue': ['blue', 'blu'],
        'brown': ['brown', 'brn'],
        'beige': ['bge', 'beige'],
        'gold': ['gold', 'gld'],
        'green': ['green', 'grn'],
        'grey': ['grey', 'gray', 'gry'],
        'orange': ['orange', 'ong'],
        'pink': ['pink', 'pnk'],
        'purple': ['purple', 'purp'],
        'red': ['red', 'rd'],
        'silver': ['silver', 'sil'],
        'turquoise': ['turquoise', 'trq'],
        'white': ['white', 'whi'],
        'yellow': ['yellow', 'yel']
       
    }

    for category, values in color_categories.items():
        for value in values:
            if value in color:
                return category

    return 'Unknown'

In [ ]:
# Apply the categorize_color function to the BIKE_COLOUR column
bicycleTheftData['BIKE_COLOUR'] = bicycleTheftData['BIKE_COLOUR'].apply(categorize_color)

In [ ]:
# Add the PRIMARY_OFFENCE and BIKE_COLOUR to the clustering dataframe - Charles
kMeanClusteringData["PRIMARY_OFFENCE"] = bicycleTheftData["PRIMARY_OFFENCE"]
kMeanClusteringData["BIKE_COLOUR"] = bicycleTheftData["BIKE_COLOUR"]

### Curtis

In [ ]:
def month_to_number(data):
    months = ["January", "February", "March","April", "May", "June",
              "July", "August", "September","October", "November", "December"]
    
    if data in months:
        return months.index(data) + 1

bicycleTheftData['OCC_MONTH'] = bicycleTheftData['OCC_MONTH'].apply(month_to_number)

In [ ]:
def day_to_number(data):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday",
            "Friday", "Saturday", "Sunday"]
    
    if data in days:
        return days.index(data) + 1

bicycleTheftData["OCC_DOW"] = bicycleTheftData["OCC_DOW"].apply(day_to_number)

In [ ]:
# Add the OCC_MONTH, OCC_DOW, PREMISES_TYPE, and BIKE_TYPE to the clustering dataframe - Charles
kMeanClusteringData["OCC_MONTH"] = bicycleTheftData["OCC_MONTH"]
kMeanClusteringData["OCC_DOW"] = bicycleTheftData["OCC_DOW"]
kMeanClusteringData["PREMISES_TYPE"] = bicycleTheftData["PREMISES_TYPE"]
kMeanClusteringData["BIKE_TYPE"] = bicycleTheftData["BIKE_TYPE"]

In [ ]:
ohe = OneHotEncoder()

encoded = pd.DataFrame(ohe.fit_transform(bicycleTheftData[['PREMISES_TYPE']]).toarray())
encoded.columns = ['Apartment','House','Commercial','Outside','Transit','Educational','Other']
bicycleTheftData = bicycleTheftData.join(encoded)


encoded = pd.DataFrame(ohe.fit_transform(bicycleTheftData[['BIKE_TYPE']]).toarray())
encoded.columns = ['BM','EL','FO','MT','OT','RC','RE','RG','SC','TA','TO','TR','UN']
bicycleTheftData = bicycleTheftData.join(encoded)

In [ ]:
bicycleTheftData = bicycleTheftData.drop(['PREMISES_TYPE'], axis=1)
bicycleTheftData = bicycleTheftData.drop(['BIKE_TYPE'], axis=1)

In [ ]:
def time_of_day(data):
    if pd.isnull(data):
        return "Unknown"
    elif data <= 4:
        return "Night"
    elif data <= 8:
        return "Dawn"
    elif data <= 11:
        return "Morning"
    elif data <= 16:
        return "Afternoon"
    elif data <= 21:
        return "Evening"
    else:
        return "Night"
    
bicycleTheftData["OCC_TOD"] = bicycleTheftData["OCC_HOUR"].apply(time_of_day)
bicycleTheftData = bicycleTheftData.drop(['OCC_HOUR'], axis=1)

In [ ]:
# Add the OCC_TOD to the clustering dataframe - Charles
kMeanClusteringData["OCC_TOD"] = bicycleTheftData["OCC_TOD"]

## Clustering

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA

In [ ]:
# Convert the nominal attributes into category
kMeanClusteringData["BIKE_MAKE"] = kMeanClusteringData["BIKE_MAKE"].astype("category")
kMeanClusteringData["DIVISION"] = kMeanClusteringData["DIVISION"].astype("category")
kMeanClusteringData["BIKE_COST_CATEGORY"] = kMeanClusteringData["BIKE_COST_CATEGORY"].astype("category")
kMeanClusteringData["BIKE_SPEED_CATEGORY"] = kMeanClusteringData["BIKE_SPEED_CATEGORY"].astype("category")
kMeanClusteringData["PRIMARY_OFFENCE"] = kMeanClusteringData["PRIMARY_OFFENCE"].astype("category")
kMeanClusteringData["BIKE_COLOUR"] = kMeanClusteringData["BIKE_COLOUR"].astype("category")
kMeanClusteringData["PREMISES_TYPE"] = kMeanClusteringData["PREMISES_TYPE"].astype("category")
kMeanClusteringData["BIKE_TYPE"] = kMeanClusteringData["BIKE_TYPE"].astype("category")
kMeanClusteringData["OCC_TOD"] = kMeanClusteringData["OCC_TOD"].astype("category")


In [ ]:
# Convert the category attributes into numeric
cat_columns = kMeanClusteringData.select_dtypes(["category"]).columns
kMeanClusteringData[cat_columns] = kMeanClusteringData[cat_columns].apply(lambda x: x.cat.codes)

In [ ]:
# Split the dataset into a train and test subset
clusteringTrain, clusteringTest = train_test_split(kMeanClusteringData, test_size=0.33, random_state=42)

### Determining K
To determine the proper value of K, multiple models are created with different K values to determine the distortion and inertia.

In [ ]:
# list to keep the values of distortion and inertia for different K
distortions = []
inertias = []
K = range(1, 10)

for k in K:
    # Build the model
    kmeanModel = KMeans(n_clusters=k, n_init='auto').fit(clusteringTrain)
    kmeanModel.fit(clusteringTrain)
    
    # Append the values
    distortions.append(sum(np.min(cdist(clusteringTrain, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / clusteringTrain.shape[0])
    inertias.append(kmeanModel.inertia_)

In [ ]:
# Plot the distortion
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

In [ ]:
# Plot the inertia
plt.plot(K, inertias, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Inertia')
plt.title('The Elbow Method using Inertia')
plt.show()

### Plotting K-Mean

In [ ]:
# reduce the number of columns to be able to plot in a 2D graph
pca = PCA(2)
reducedClusteringTrainData = pca.fit_transform(clusteringTrain)

In [ ]:
# create the KMeans algorithm and cluster the training data
kmeans = KMeans(n_clusters= 4, n_init="auto")
trainLablels = kmeans.fit_predict(reducedClusteringTrainData)

In [ ]:
#Getting unique labels
u_labels = np.unique(trainLablels)
 
#plotting the results:
for i in u_labels:
    plt.scatter(reducedClusteringTrainData[trainLablels == i , 0] , reducedClusteringTrainData[trainLablels == i , 1] , label= i)
plt.legend()
plt.show()

In [ ]:
#Getting the Centroids
centroids = kmeans.cluster_centers_
u_labels = np.unique(trainLablels)
 
#plotting the results:
for i in u_labels:
    plt.scatter(reducedClusteringTrainData[trainLablels == i , 0] , reducedClusteringTrainData[trainLablels == i , 1] , label= i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()

### Compare to Test

In [ ]:
# reduce the number of columns to be able to plot in a 2D graph
reducedClusteringTestData = pca.fit_transform(clusteringTest)

In [ ]:
# create the KMeans algorithm and cluster the training data
kmeans = KMeans(n_clusters= 4, n_init="auto")
testLabels = kmeans.fit_predict(reducedClusteringTestData)

In [ ]:
#Getting unique labels
u_labels = np.unique(testLabels)
 
#plotting the results:
for i in u_labels:
    plt.scatter(reducedClusteringTestData[testLabels == i , 0] , reducedClusteringTestData[testLabels == i , 1] , label = i)
plt.legend()
plt.show()

In [ ]:
#Getting the Centroids
centroids = kmeans.cluster_centers_
u_labels = np.unique(testLabels)
 
#plotting the results:
for i in u_labels:
    plt.scatter(reducedClusteringTestData[testLabels == i , 0] , reducedClusteringTestData[testLabels == i , 1] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()

### Evaluate

In [ ]:
# add the cluster number to each instance
clusteringTrain["CLUSTER"] = trainLablels

# save the description of each cluster to csv
for x in range(0,4):
    clusteringTrain.loc[clusteringTrain["CLUSTER"] == x].describe().to_csv("TRAINCLUSTER{0}.csv".format(x))

In [ ]:
# add the cluster number to each instance
clusteringTest["CLUSTER"] = testLabels

# save the description of each cluster to csv
for x in range(0,4):
    print(clusteringTest.loc[clusteringTest["CLUSTER"] == x].describe())